In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 载入数据集
mnist = input_data.read_data_sets(r"C:\Users\zdwxx\Downloads\Compressed\MNIST_data", one_hot=True)

# 运行次数
max_steps = 550 * 21

# 图片数量
image_num = 3000

# 定义会话
sess = tf.Session()

# 文件路径
DIR = "C:/Tensorflow/"

# 载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]), 
                        trainable=False, name="embedding")

# 定义一个参数概要
def varible_summaries(var, name="summary"):
    
    with tf.name_scope(name):
        
        mean = tf.reduce_mean(var)
        tf.summary.scalar("mean", mean) # 平均值
        with tf.name_scope("stddev"):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar("stddev", stddev) # 标准差
        tf.summary.scalar("max", tf.reduce_max(var)) #最大值
        tf.summary.scalar("min", tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram", var) # 直方图

# 命名空间
with tf.name_scope("input"):
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 784], name="x-input")
    y = tf.placeholder(tf.float32, [None, 10], name="y-input")        

# 显示图片
with tf.name_scope("input_reshape"):
    
    image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
    tf.summary.image("input", image_shaped_input, 10)
    

with tf.name_scope("layer"):
        #创建一个简单的神经网络
    # 初始化权值函数
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.1) # 生成一个截断的正态分布
        return tf.Variable(initial)

    # 初始化偏置值函数
    def bias_variable(shape):
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    # 卷积层
    def conv2d(x, W):
    #   x input tensor of shape:`[filter_height * filter_wid3th * in_channels, output_channels]`.
    # `[filter_height, filter_width, in_channels, out_channels]`
    # `strides[0] = strides[3] = 1,strides[1]->x方向上的补偿，[2]->y方向上的补偿
    # padding: A string from:"SAME", "VAILD"
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

    # 池化层
    def max_pool_2x2(x):
    # ksize[1,x,y,1]
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

    # 改变x的格式转为4D的向量[batch, in_height, in_widch, in_channels]
    x_image = tf.reshape(x, [-1, 28, 28, 1]) # [批次，高，宽，通道]

    # 初始化第一个卷积层的权值和偏置值
    W_conv1 = weight_variable([5, 5, 1, 32]) # 5*5的采样窗口，32个卷积核从1个平面抽取特征

    b_conv1 = bias_variable([32]) # 每一个卷积核的偏置值



    # 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) # 进行max_pool


    # 初始化第二个卷积层的权值和偏置值
    W_conv2 = weight_variable([5, 5, 32, 64]) # 5*5的采样窗口，64个卷积核从32个平面抽取特征

    b_conv2 = bias_variable([64]) # 每一个卷积核的偏置值


    # 把x_image的权值进行卷积，加上偏置值，然后应用于relu激活函数
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) # 进行max_pool


    # 28*28的图片第一次卷积后还是28*28，第一次池化后变成了14*14
    # 第二次卷积后为14*14，第二次池化后变成了7*7
    # 通过上面的操作后得到了64张7*7的平面

    # 初始化第一个全连接层的权值
    W_fc1 = weight_variable([7*7*64, 1024])

    b_fc1 = bias_variable([1024])


    # 把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

    # 求出第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#     # 使用dropout防止过拟合
#     h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


    # 初始化第二个全连接层
    W_fc2 = weight_variable([1024, 10])

    b_fc2 = bias_variable([10])


    # 计算输出
    prediction = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

# 二次代价函数

# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    tf.summary.scalar("loss", loss)
    
# Adam
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

with tf.name_scope("accuracy"):
    # 结果存放在一个布尔型列表中
    with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1)) #argmax返回1维张量中最大的值所在的位置
    # 求准确率
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))#cast转换类型，True->1.0, False->0.0
        tf.summary.scalar("accuracy", accuracy)

# 产生 metadata文件        
if tf.gfile.Exists(DIR + "projector/projector/metadata.tsv"):
    tf.gfile.DeleteRecursively(DIR + "projector/projector/metadata.tsv")

with open(DIR + "projector/projector/metadata.tsv", "w") as f:
    lables = sess.run(tf.argmax(mnist.test.labels[:], 1))
    for i in range(image_num):
        f.write(str(lables[i]) + "\n")

# 合并所有的summary
merged = tf.summary.merge_all()


projector_writer = tf.summary.FileWriter(DIR + "projector/projector", sess.graph)
saver = tf.train.Saver()
config = projector.ProjectorConfig()
embed = config.embeddings.add()
embed.tensor_name = embedding.name
embed.metadata_path = DIR + "projector/projector/metadata.tsv"
embed.sprite.image_path = DIR + "projector/data/mnist_10k_sprite.png"
embed.sprite.single_image_dim.extend([28, 28])
projector.visualize_embeddings(projector_writer, config)
    

for i in range(max_steps):

    batch_xs, batch_ys = mnist.train.next_batch(100) #类似于read，一次读取100张图片
    run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    run_metadata = tf.RunMetadata()
    sess.run(train_step, feed_dict={x : batch_xs, y : batch_ys})
    projector_writer.add_run_metadata(run_metadata, "step%03d" % i)

    
    if i % 550 == 0:
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        summary = sess.run(merged, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        projector_writer.add_summary(summary, i)
        print("第", i, "个周期", "准确率是", acc)

saver.save(sess, DIR + "projector/projector/a_model.ckpt")
projector_writer.close()
sess.close()

Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\train-labels-idx1-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-images-idx3-ubyte.gz
Extracting C:\Users\zdwxx\Downloads\Compressed\MNIST_data\t10k-labels-idx1-ubyte.gz
第 0 个周期 准确率是 0.1543
第 550 个周期 准确率是 0.765
第 1100 个周期 准确率是 0.7856
第 1650 个周期 准确率是 0.7886
第 2200 个周期 准确率是 0.8763
第 2750 个周期 准确率是 0.8876
第 3300 个周期 准确率是 0.9844
第 3850 个周期 准确率是 0.9859
第 4400 个周期 准确率是 0.9859
第 4950 个周期 准确率是 0.9876
第 5500 个周期 准确率是 0.9873
第 6050 个周期 准确率是 0.987
第 6600 个周期 准确率是 0.9883
第 7150 个周期 准确率是 0.9895
第 7700 个周期 准确率是 0.9888
第 8250 个周期 准确率是 0.9887
第 8800 个周期 准确率是 0.9898
第 9350 个周期 准确率是 0.9882
第 9900 个周期 准确率是 0.9907
第 10450 个周期 准确率是 0.9906
第 11000 个周期 准确率是 0.9905
